# "Intuitively understanding Python decorators"
> "Syntactic sugar and functional implementation of decorators"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [Python, functional programming]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

## Functions

In [3]:
def add_one(number): return number+1

add_one(2)

3

### First-class objects

In [4]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

In [5]:
greet_bob(say_hello)

'Hello Bob'

In [6]:
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'